## get parcel data per apn
provides useful info, especially for development.

does not have any ownership info

In [5]:
import pandas as pd
import time,pathlib,re
import scipy as sp
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

In [84]:
# get a list of apns
if(1):
    sheetname = "data"
    df = pd.read_excel("C:\\Users\\raysa\\docs\\real estate\\sd-tax-auction\\2023-sd-tax-auction\\tax-auction-data.xlsx",sheet_name=sheetname)
    df = df[(df["Auction Type"]!="Timeshare Property") & (df["Canceled"].isnull())]
    apns = df["APN"]
else:
    apns= ["123563454"]

df.set_index("APN",inplace=True)

N= np.size(apns)
N

119

https://sites.google.com/chromium.org/driver/downloads?authuser=0

In [49]:
resultFolder = pathlib.Path("C:/work/scratch/temp_sangis")
#resultFolder.mkdir(parents=True, exist_ok=True)


tmpFileName= "{APN}.sangis.html"


In [58]:
help(driver.get)

Help on method get in module selenium.webdriver.remote.webdriver:

get(url: str) -> None method of selenium.webdriver.chrome.webdriver.WebDriver instance
    Loads a web page in the current browser session.



In [59]:
idx = 0
for apn in apns:
    print("%d (%2.2f) -->%d"%(idx,100.0 *(idx+1)/N,apn));
    idx+=1
    # download the executable
    driver  = webdriver.Chrome("C:\work\chrome_driver_110\\chromedriver.exe")
    driver.implicitly_wait(30) # seconds
    url =  'https://gissd.sandag.org/plt/ParcelRpt.aspx?APN={APN}'.format(APN=apn)
    driver.get(url)
    time.sleep(30)
    data = driver.page_source
    driver.close()

    with resultFolder.joinpath(tmpFileName.format(APN=apn)).open('w') as f:
        f.write(data);
print("done")   

0 (0.84) -->1092702600


C:\Users\raysa\AppData\Local\Temp\ipykernel_6948\2249667104.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver  = webdriver.Chrome("C:\work\chrome_driver_110\\chromedriver.exe")


1 (1.68) -->1223202100
2 (2.52) -->1250904206
3 (3.36) -->1481603200
4 (4.20) -->1491912100
5 (5.04) -->1511510200
6 (5.88) -->1623440600
7 (6.72) -->1681800300
8 (7.56) -->1800806700
9 (8.40) -->1873602300
10 (9.24) -->2090412501
11 (10.08) -->2162100345
12 (10.92) -->2162903905
13 (11.76) -->2176500400
14 (12.61) -->2212500500
15 (13.45) -->2251713700
16 (14.29) -->2310707706
17 (15.13) -->2886500531
18 (15.97) -->3133423000
19 (16.81) -->3162400103
20 (17.65) -->3173630300
21 (18.49) -->3465404616
22 (19.33) -->3821401900
23 (20.17) -->4212302143
24 (21.01) -->4450412500
25 (21.85) -->4476520200
26 (22.69) -->4667311500
27 (23.53) -->4803031100
28 (24.37) -->4850310900
29 (25.21) -->4971152200
30 (26.05) -->4993748400
31 (26.89) -->5011771300
32 (27.73) -->5012330300
33 (28.57) -->5051121800
34 (29.41) -->5112802700
35 (30.25) -->5130730700
36 (31.09) -->5396012624
37 (31.93) -->5432540500
38 (32.77) -->5453021300
39 (33.61) -->5454521500
40 (34.45) -->5754700100
41 (35.29) -->58017

# parse html pages saved

now get all the data from html pages and create a composite csv file with one row per apn

In [60]:
mapper = {
#'Property Characteristics:',
'Baths:'                      :3,
'Assessor Land:'              :2,
'Improvements:'               :7,
#'Add. Areas:',
#'Tot. Living Area:',
'Assessor Total:'              :9 ,
'Tax Status:'                 :14,  
'Acreage:'                    :15,
'Bedrooms:'                   :18,
'Tax Rate Area:'              :17,      
 #'District Information:',
 #'City Council:',
 #'Board of Supervisors:',
 #'School District:',
 #'School Union:',
 #'Congress:',
 #'Fire Protection:',
 #'Senate:',
 #'Assembly:',
 #'Community College:',
 #'Planning Areas:',
 #'Census Tract:',
'Subregional Area:'           :67,
 #'Major Statistical Area:',
 #'Community Planning Area:',
'Land Use:'                   :slice(74,75),
'Existing (LU):'              :slice(78,79),
 #'Ecology:',
 #'Vegetation Type (County):',
 #'Vegetation Type (Western Region 2012):',
'Flood Zone/Flood Plain:'     :[89,91]
 #'Geology Type:',
 #'Soil Type:'
}

def processHtml(apn):
    with resultFolder.joinpath(tmpFileName.format(APN=apn)).open('r') as f:
        #try:    
            #f=open(file,'r')
        data = f.read()
            #f.close()
    #except:
    #    print("cant find html for :%s"%(file))
    #    return -1
    soup = BeautifulSoup(data)
  # save a pretty version
    if(0):
        f = open("test.html",'w')
        f.write(soup.prettify());
        f.close()
                               
    ELEM_SEPERATOR = '|'
    ID_IDENTIFIER  = ':'
    s = soup.get_text(ELEM_SEPERATOR)
    startI   = int(s.find("Property Characteristics"))
    endI     = int(s.find("Data Disclaimer"))
    content  = s[startI:endI]
    contentS =content.split(ELEM_SEPERATOR)
    contentId=[c.strip() for c in contentS if ID_IDENTIFIER in c]
    return contentS;                               

In [91]:



df["LU code"]         =''
df["LU Description"]  =''
df["PLU Code"]        =''
df["PLU Description"] =''
df["Land type info"]  =''
for key, idx in mapper.items():
    df[key] = ''


for apn in apns:
    content = processHtml(apn)
    if(content==-1):continue
    #str = "%s,%s,"%(apn,sheetname);
    e = {}
    for key, idx in mapper.items():
        if(isinstance(idx,slice)):
            entry =" ".join(content[idx])
        elif(isinstance(idx,list)):
            entry =" ".join([content[i] for i in idx])
        else:
            entry =content[idx]
        df.loc[apn,key]=entry
    
    flat = "".join(content);
    found = re.search("Land Use.*?LU\)\:([0-9]*)(.*?)\(PLU\)([0-9]*)(.*?)Ecology",flat)
    #str +=",".join(found.groups())+","+found.group (0);
    df.loc[apn,"LU code"]         = found.groups(0)[0]
    df.loc[apn,"LU Description"]  = found.groups(0)[1]
    df.loc[apn,"PLU Code"]        = found.groups(0)[2]
    df.loc[apn,"PLU Description"] = found.groups(0)[3]
    df.loc[apn,"Land type info"]  = found.group(0)
    
df.to_excel("data.xlsx")

In [94]:
df.to_excel(resultFolder/"data.xlsx")